# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import random
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
test = pd.read_csv('../input/shopee-product-matching/test.csv')
sample = pd.read_csv('../input/shopee-product-matching/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
#計算圖片有幾種lebel,一個label最大跟最小
vc = dict(train.label_group.value_counts())
counts = list(set(vc.values()))
print('min = ', min(counts), ", max = ", max(counts), ', all = ', sum(vc.values()))

In [ ]:
train.info()

In [ ]:
#看csv 裡 有幾種不重複的值
for i in train.columns:
    print(i, len(np.unique(train[i])))

圖片只給32412種,代表重複的有34250-32412

In [ ]:
#處裡資料(image)
train=train.drop(['image_phash','title'], axis=1)
train.head()

In [ ]:
DATA_PATH = '/../input/shopee-product-matching'
train['image_path'] = DATA_PATH + 'train_images/' + train['image']
train.head()

In [ ]:
dftrain=train.drop(['posting_id','image_path'], axis=1)
dftrain.head()

In [ ]:
dftest=test.drop(['posting_id','image_phash','title'], axis=1)
dftest.head()

"""
This kernel takes about 12 min to run:

* Image preprocessing: ~ 8 min
* Model training: ~ 4 min

Since MobileNet v2 has only 2.2M parameters, it trains faster than
ResNet-50 (26M) and ResNext-101 (84M); as a tradeoff, the potential 
accuracy becomes lower. We will finetune the pretrained ImageNet weights.
"""
import os
import json
import math

import cv2
import numpy as np
import pandas as pd
from keras import layers, optimizers
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from sklearn.model_selection import train_test_split


def pad_and_resize(image_path, pad=True, desired_size=224):
    def get_pad_width(im, new_shape, is_rgb=True):
        pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
        t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
        l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
        if is_rgb:
            pad_width = ((t,b), (l,r), (0, 0))
        else:
            pad_width = ((t,b), (l,r))
        return pad_width
        
    img = cv2.imread(image_path)
    
    if pad:
        pad_width = get_pad_width(img, max(img.shape))
        padded = np.pad(img, pad_width=pad_width, mode='constant', constant_values=0)
    else:
        padded = img
    
    padded = cv2.cvtColor(padded, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(padded, (desired_size,)*2).astype('uint8')
    
    return resized


def build_model():
    mobilenet = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224,224,3)
    )
    model = Sequential()
    model.add(mobilenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers.Adam(lr=0.00002),
        metrics=['accuracy']
    )
    
    return model


# Load Labels
train_df = dftrain
test_df = dftest

# Load Images
train_resized_imgs = []
test_resized_imgs = []

for image in train_df['image']:
    img = pad_and_resize(f'../input/shopee-product-matching/train_images/{image}')
    train_resized_imgs.append(img)

for image in test_df['image']:
    img = pad_and_resize(f'../input/shopee-product-matching/test_images/{image}')
    test_resized_imgs.append(img)

x_test = np.stack(test_resized_imgs)

# Split Data
x_train, x_val, y_train, y_val = train_test_split(
    np.stack(train_resized_imgs), 
    pd.get_dummies(train_df['label_group']).values, 
    test_size=0.15, 
    random_state=2019
)

# Train Model    
model = build_model()
checkpoint = ModelCheckpoint('model.h5', save_best_only=True)
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    verbose=2,
    callbacks=[checkpoint],
    validation_data=(x_val, y_val)
)

# Submission
model.load_weights('model.h5')
y_test = model.predict(x_test)
test_df['label_group'] = y_test.argmax(axis=1)
#test_df.to_csv('submission.csv',index=False)
print(test_df)